# Machine Learning

In this file, instructions how to approach the challenge can be found.

We are going to work on different types of Machine Learning problems:

- **Regression Problem**: The goal is to predict delay of flights.
- **(Stretch) Multiclass Classification**: If the plane was delayed, we will predict what type of delay it is (will be).
- **(Stretch) Binary Classification**: The goal is to predict if the flight will be cancelled.

## Main Task: Regression Problem

The target variable is **ARR_DELAY**. We need to be careful which columns to use and which don't. For example, DEP_DELAY is going to be the perfect predictor, but we can't use it because in real-life scenario, we want to predict the delay before the flight takes of --> We can use average delay from earlier days but not the one from the actual flight we predict.  

For example, variables **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY** shouldn't be used directly as predictors as well. However, we can create various transformations from earlier values.

We will be evaluating your models by predicting the ARR_DELAY for all flights **1 week in advance**.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro
from random import sample 

In C:\Users\Beasty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Beasty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Beasty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Beasty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Beasty\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_tes

In [2]:
#For the purpose of Analysis we have taken flight for DL carrier in 2019.
df = pd.read_csv('UA flights 2019.csv', low_memory=False, parse_dates=[0])

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df.columns

Index(['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name'],
      dtype='object')

In [5]:
df.tail()

,fl_date,mkt_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
1571399,2019-12-31,UA,3769,N454AW,3769,10408,ATW,"Appleton, WI",13930,ORD,"Chicago, IL",1155,1341.0,106.0,25.0,1406.0,1444.0,13.0,1320,1457.0,97.0,0.0,NaN,0.0,N,85.0,76.0,38.0,1.0,161.0,2.0,0.0,0.0,0.0,95.0,NaN,NaN,NaN,NaN
1571400,2019-12-31,UA,3768,N444ZW,3768,13342,MKE,"Milwaukee, WI",13930,ORD,"Chicago, IL",936,1005.0,29.0,29.0,1034.0,1102.0,37.0,1045,1139.0,54.0,0.0,NaN,0.0,N,69.0,94.0,28.0,1.0,67.0,12.0,0.0,25.0,0.0,17.0,NaN,NaN,NaN,NaN
1571401,2019-12-31,UA,3767,N433AW,3767,11193,CVG,"Cincinnati, OH",13930,ORD,"Chicago, IL",1726,1720.0,-6.0,11.0,1731.0,1720.0,10.0,1803,1730.0,-33.0,0.0,NaN,0.0,N,97.0,70.0,49.0,1.0,264.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1571402,2019-12-31,UA,3766,N427ZW,3766,10257,ALB,"Albany, NY",13930,ORD,"Chicago, IL",1300,1254.0,-6.0,11.0,1305.0,1359.0,5.0,1445,1404.0,-41.0,0.0,NaN,0.0,N,165.0,130.0,114.0,1.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1571403,2019-12-31,UA,3765,N416AW,3765,13930,ORD,"Chicago, IL",10408,ATW,"Appleton, WI",1405,1445.0,40.0,17.0,1502.0,1537.0,4.0,1514,1541.0,27.0,0.0,NaN,0.0,N,69.0,56.0,35.0,1.0,161.0,27.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [6]:
df.dtypes

fl_date                datetime64[ns]
mkt_carrier                    object
mkt_carrier_fl_num              int64
tail_num                       object
op_carrier_fl_num               int64
origin_airport_id               int64
origin                         object
origin_city_name               object
dest_airport_id                 int64
dest                           object
dest_city_name                 object
crs_dep_time                    int64
dep_time                      float64
dep_delay                     float64
taxi_out                      float64
wheels_off                    float64
wheels_on                     float64
taxi_in                       float64
crs_arr_time                    int64
arr_time                      float64
arr_delay                     float64
cancelled                     float64
cancellation_code              object
diverted                      float64
dup                            object
crs_elapsed_time              float64
actual_elaps

In [7]:
df = df.sort_values(['fl_date'])

In [8]:
df.shape

(1571404, 39)

In [9]:
df.tail()

,fl_date,mkt_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
1568912,2019-12-31,UA,3534,N861RW,3534,12339,IND,"Indianapolis, IN",12266,IAH,"Houston, TX",1705,1703.0,-2.0,37.0,1740.0,1853.0,4.0,1854,1857.0,3.0,0.0,NaN,0.0,N,169.0,174.0,133.0,1.0,844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1568913,2019-12-31,UA,3535,N862RW,3535,13871,OMA,"Omaha, NE",13930,ORD,"Chicago, IL",905,857.0,-8.0,12.0,909.0,1018.0,14.0,1053,1032.0,-21.0,0.0,NaN,0.0,N,108.0,95.0,69.0,1.0,416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1568914,2019-12-31,UA,3536,N732YX,3536,11278,DCA,"Washington, DC",11618,EWR,"Newark, NJ",1955,1949.0,-6.0,12.0,2001.0,2040.0,26.0,2130,2106.0,-24.0,0.0,NaN,0.0,N,95.0,77.0,39.0,1.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1568902,2019-12-31,UA,3523,N650RW,3523,11618,EWR,"Newark, NJ",13198,MCI,"Kansas City, MO",1110,1117.0,7.0,16.0,1133.0,1314.0,3.0,1346,1317.0,-29.0,0.0,NaN,0.0,N,216.0,180.0,161.0,1.0,1092.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1571403,2019-12-31,UA,3765,N416AW,3765,13930,ORD,"Chicago, IL",10408,ATW,"Appleton, WI",1405,1445.0,40.0,17.0,1502.0,1537.0,4.0,1514,1541.0,27.0,0.0,NaN,0.0,N,69.0,56.0,35.0,1.0,161.0,27.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [10]:
# df_taxi_out=df[['origin_airport_id', 'fl_date',  'taxi_out']].groupby(['origin_airport_id', 'fl_date']).rolling(30 
#                                                                         ).agg({'taxi_out':'mean'})

In [11]:
df_taxi_out=df[['origin_airport_id', 'fl_date',  'taxi_out']].groupby(['origin_airport_id', 'fl_date']).mean().reset_index()

In [12]:
df_taxi_out

,origin_airport_id,fl_date,taxi_out
0,10135,2019-01-01,12.000000
1,10135,2019-01-02,13.000000
2,10135,2019-01-03,12.000000
3,10135,2019-01-04,13.500000
4,10135,2019-01-05,19.500000
...,...,...,...
84473,16869,2019-12-27,22.333333
84474,16869,2019-12-28,17.000000
84475,16869,2019-12-29,29.666667
84476,16869,2019-12-30,26.333333


In [13]:
df_taxi_out_roll=df_taxi_out.groupby(['origin_airport_id']).rolling(30, on='fl_date').agg({'taxi_out':'mean'}).reset_index()

In [14]:
df_taxi_out_roll

,origin_airport_id,fl_date,taxi_out
0,10135,2019-01-01,NaN
1,10135,2019-01-02,NaN
2,10135,2019-01-03,NaN
3,10135,2019-01-04,NaN
4,10135,2019-01-05,NaN
...,...,...,...
84473,16869,2019-12-27,NaN
84474,16869,2019-12-28,NaN
84475,16869,2019-12-29,31.144444
84476,16869,2019-12-30,28.922222


In [15]:
df_taxi_out_roll.rename(columns={'taxi_out':'30d taxi out'}, inplace=True)

In [19]:
df=df.merge(df_taxi_out_roll, on=['origin_airport_id', 'fl_date' ] , how='left')

In [22]:
#Calculating taxi_in time
df_taxi_in=df[['dest_airport_id', 'fl_date',  'taxi_in']].groupby(['dest_airport_id', 'fl_date']).mean().reset_index()
df_taxi_in

,dest_airport_id,fl_date,taxi_in
0,10135,2019-01-01,6.000000
1,10135,2019-01-02,4.500000
2,10135,2019-01-03,4.000000
3,10135,2019-01-04,4.000000
4,10135,2019-01-05,5.000000
...,...,...,...
84336,16869,2019-12-27,11.000000
84337,16869,2019-12-28,5.666667
84338,16869,2019-12-29,4.666667
84339,16869,2019-12-30,6.666667


In [23]:
df_taxi_in_roll=df_taxi_in.groupby(['dest_airport_id']).rolling(30, on='fl_date').agg({'taxi_in':'mean'}).reset_index()
df_taxi_in_roll

,dest_airport_id,fl_date,taxi_in
0,10135,2019-01-01,NaN
1,10135,2019-01-02,NaN
2,10135,2019-01-03,NaN
3,10135,2019-01-04,NaN
4,10135,2019-01-05,NaN
...,...,...,...
84336,16869,2019-12-27,NaN
84337,16869,2019-12-28,NaN
84338,16869,2019-12-29,NaN
84339,16869,2019-12-30,7.283333


In [24]:
df_taxi_in_roll.rename(columns={'taxi_in':'30d taxi in'}, inplace=True)

In [25]:
df=df.merge(df_taxi_in_roll, on=['dest_airport_id', 'fl_date' ] , how='left')

In [27]:
df.tail()

,fl_date,mkt_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,30d taxi out,30d taxi in
1571399,2019-12-31,UA,3534,N861RW,3534,12339,IND,"Indianapolis, IN",12266,IAH,"Houston, TX",1705,1703.0,-2.0,37.0,1740.0,1853.0,4.0,1854,1857.0,3.0,0.0,NaN,0.0,N,169.0,174.0,133.0,1.0,844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.367877,8.386432
1571400,2019-12-31,UA,3535,N862RW,3535,13871,OMA,"Omaha, NE",13930,ORD,"Chicago, IL",905,857.0,-8.0,12.0,909.0,1018.0,14.0,1053,1032.0,-21.0,0.0,NaN,0.0,N,108.0,95.0,69.0,1.0,416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.426397,11.688253
1571401,2019-12-31,UA,3536,N732YX,3536,11278,DCA,"Washington, DC",11618,EWR,"Newark, NJ",1955,1949.0,-6.0,12.0,2001.0,2040.0,26.0,2130,2106.0,-24.0,0.0,NaN,0.0,N,95.0,77.0,39.0,1.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.009218,13.432760
1571402,2019-12-31,UA,3523,N650RW,3523,11618,EWR,"Newark, NJ",13198,MCI,"Kansas City, MO",1110,1117.0,7.0,16.0,1133.0,1314.0,3.0,1346,1317.0,-29.0,0.0,NaN,0.0,N,216.0,180.0,161.0,1.0,1092.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.253805,5.892857
1571403,2019-12-31,UA,3765,N416AW,3765,13930,ORD,"Chicago, IL",10408,ATW,"Appleton, WI",1405,1445.0,40.0,17.0,1502.0,1537.0,4.0,1514,1541.0,27.0,0.0,NaN,0.0,N,69.0,56.0,35.0,1.0,161.0,27.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,23.927712,NaN


In [31]:
#Now calculating trafic per airport per day. Also will calculate 30d rolling average.
count_flight=df[['origin_airport_id', 'fl_date', 'mkt_carrier']].groupby(['origin_airport_id', 'fl_date'
                                                                         ]).count().reset_index()

In [32]:
count_flight

,origin_airport_id,fl_date,mkt_carrier
0,10135,2019-01-01,2
1,10135,2019-01-02,2
2,10135,2019-01-03,2
3,10135,2019-01-04,2
4,10135,2019-01-05,2
...,...,...,...
84473,16869,2019-12-27,3
84474,16869,2019-12-28,3
84475,16869,2019-12-29,3
84476,16869,2019-12-30,3


In [34]:
count_flights_roll= count_flight.groupby(['origin_airport_id']).rolling(30, on='fl_date'
                                                                       ).agg({'mkt_carrier':'mean'}).reset_index()
count_flights_roll

,origin_airport_id,fl_date,mkt_carrier
0,10135,2019-01-01,NaN
1,10135,2019-01-02,NaN
2,10135,2019-01-03,NaN
3,10135,2019-01-04,NaN
4,10135,2019-01-05,NaN
...,...,...,...
84473,16869,2019-12-27,2.866667
84474,16869,2019-12-28,2.933333
84475,16869,2019-12-29,2.933333
84476,16869,2019-12-30,2.966667


In [36]:
count_flights_roll.rename(columns={'mkt_carrier':'30d roll flights count'}, inplace=True)

In [40]:
df=df.merge(count_flights_roll, on=['origin_airport_id', 'fl_date' ] , how='left')

In [41]:
df

,fl_date,mkt_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,cancelled,cancellation_code,diverted,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name,30d taxi out,30d taxi in,30d roll flights count
0,2019-01-01,UA,5611,N947SW,5611,14689,SBA,"Santa Barbara, CA",12892,LAX,"Los Angeles, CA",1915,1904.0,-11.0,12.0,1916.0,1942.0,12.0,2017,1954.0,-23.0,0.0,NaN,0.0,N,62.0,50.0,26.0,1.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01,UA,4902,N11181,4902,11618,EWR,"Newark, NJ",13485,MSN,"Madison, WI",918,925.0,7.0,21.0,946.0,1112.0,3.0,1106,1115.0,9.0,0.0,NaN,0.0,N,168.0,170.0,146.0,1.0,799.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01,UA,4901,N14179,4901,12264,IAD,"Washington, DC",10990,CHO,"Charlottesville, VA",1745,1744.0,-1.0,17.0,1801.0,1823.0,3.0,1841,1826.0,-15.0,0.0,NaN,0.0,N,56.0,42.0,22.0,1.0,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01,UA,4900,N16183,4900,11618,EWR,"Newark, NJ",12264,IAD,"Washington, DC",1855,1852.0,-3.0,19.0,1911.0,2003.0,4.0,2024,2007.0,-17.0,0.0,NaN,0.0,N,89.0,75.0,52.0,1.0,212.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01,UA,4880,N16147,4880,11618,EWR,"Newark, NJ",11066,CMH,"Columbus, OH",802,757.0,-5.0,24.0,821.0,955.0,9.0,1004,1004.0,0.0,0.0,NaN,0.0,N,122.0,127.0,94.0,1.0,463.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1571399,2019-12-31,UA,3534,N861RW,3534,12339,IND,"Indianapolis, IN",12266,IAH,"Houston, TX",1705,1703.0,-2.0,37.0,1740.0,1853.0,4.0,1854,1857.0,3.0,0.0,NaN,0.0,N,169.0,174.0,133.0,1.0,844.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18.367877,8.386432,26.666667
1571400,2019-12-31,UA,3535,N862RW,3535,13871,OMA,"Omaha, NE",13930,ORD,"Chicago, IL",905,857.0,-8.0,12.0,909.0,1018.0,14.0,1053,1032.0,-21.0,0.0,NaN,0.0,N,108.0,95.0,69.0,1.0,416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.426397,11.688253,16.000000
1571401,2019-12-31,UA,3536,N732YX,3536,11278,DCA,"Washington, DC",11618,EWR,"Newark, NJ",1955,1949.0,-6.0,12.0,2001.0,2040.0,26.0,2130,2106.0,-24.0,0.0,NaN,0.0,N,95.0,77.0,39.0,1.0,199.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.009218,13.432760,32.200000
1571402,2019-12-31,UA,3523,N650RW,3523,11618,EWR,"Newark, NJ",13198,MCI,"Kansas City, MO",1110,1117.0,7.0,16.0,1133.0,1314.0,3.0,1346,1317.0,-29.0,0.0,NaN,0.0,N,216.0,180.0,161.0,1.0,1092.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.253805,5.892857,319.766667


### Feature Engineering

Feature engineering will play a crucial role in this problems. We have only very little attributes so we need to create some features that will have some predictive power.

- weather: we can use some weather API to look for the weather in time of the scheduled departure and scheduled arrival.
- statistics (avg, mean, median, std, min, max...): we can take a look at previous delays and compute descriptive statistics
- airports encoding: we need to think about what to do with the airports and other categorical variables
- time of the day: the delay probably depends on the airport traffic which varies during the day.
- airport traffic
- unsupervised learning as feature engineering?
- **what are the additional options?**: Think about what we could do more to improve the model.

### Feature Selection / Dimensionality Reduction

We need to apply different selection techniques to find out which one will be the best for our problems.

- Original Features vs. PCA conponents?

### Modeling

Use different ML techniques to predict each problem.

- linear / logistic / multinomial logistic regression
- Naive Bayes
- Random Forest
- SVM
- XGBoost
- The ensemble of your own choice

### Evaluation

You have data from 2018 and 2019 to develop models. Use different evaluation metrics for each problem and compare the performance of different models.

You are required to predict delays on **out of sample** data from **first 7 days (1st-7th) of January 2020** and to share the file with LighthouseLabs. Sample submission can be found in the file **_sample_submission.csv_**

======================================================================
## Stretch Tasks

### Multiclass Classification

The target variables are **CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY**. We need to do additional transformations because these variables are not binary but continuos. For each flight that was delayed, we need to have one of these variables as 1 and others 0.

It can happen that we have two types of delays with more than 0 minutes. In this case, take the bigger one as 1 and others as 0.

### Binary Classification

The target variable is **CANCELLED**. The main problem here is going to be huge class imbalance. We have only very little cancelled flights with comparison to all flights. It is important to do the right sampling before training and to choose correct evaluation metrics.